In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


2023-12-07 14:29:14.986562: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 14:29:15.286123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 14:29:15.286223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 14:29:15.335932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 14:29:15.439312: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 14:29:15.441256: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
df = pd.read_csv('../raw_data/CLEAR Corpus 6.01 - CLEAR Corpus 6.01.csv')
df = df.drop(columns=['Last Changed', 'URL', 'Anthology', 'MPAA \n#Max', 'Pub Year', 'MPAA\n#Avg', 'License', 'British Words', 'firstPlace_pred', 'secondPlace_pred', 'thirdPlace_pred',
       'fourthPlace_pred', 'fifthPlace_pred', 'sixthPlace_pred', 'ID', 'Author', 'Title', 'Source', 'Category', 'Location', 'MPAA\nMax', 'BT s.e.', 'Kaggle split'])


In [4]:
df['len_char'] = df['Excerpt'].apply(lambda x: len(x))

def count_punctuation(x):
    punc = string.punctuation
    count=0
    for char in x:
        if char in punc:
            count+=1
    return count/ len(x)

df['punctuation_ratio'] = df['Excerpt'].apply(count_punctuation)



def sentence_len(x):
    x = x.replace("?",'.').replace("!",'.')
    sentences = x.split('.')
    sentences = [sentence for sentence in sentences if len(sentence) > 0]
    sent_len = [len(sentence.split()) for sentence in sentences]
    return np.mean(sent_len)

df['avg_sentence_len'] = df['Excerpt'].apply(sentence_len)


def avg_word_lenght(x):
    punctuation = string.punctuation
    for punc in punctuation:
        x = x.replace(punc,' ')
    words = x.split(' ')
    words = [word for word in words if len(word) > 0]
    word_len = [len(word) for word in words]
    return np.mean(word_len)

df['avg_word_len'] = df['Excerpt'].apply(avg_word_lenght)

df['avg_word_sent'] = df['Google\nWC']/df['Sentence\nCount v1']


In [5]:
quantiles = df['BT Easiness'].quantile([0, 0.1667, 0.3333, 0.5, 0.6667, 0.8333, 1]).tolist()

# Correct number of labels for 6 bins
labels_dict = {'very hard':0, 'hard':1, 'moderately hard':2, 'acceptable':3, 'easy':4, 'very easy':5}

# Using 'quantiles' for bins and including 6 labels
df['BT Easiness'] = pd.cut(
    x=df['BT Easiness'],
    bins=quantiles,
    labels=labels_dict.values(),
    include_lowest=True
)


In [6]:
X = df.drop(columns=['BT Easiness'])
y = df['BT Easiness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [7]:
X_train_text = X_train['Excerpt'].values
X_test_text = X_test['Excerpt'].values


In [9]:
# Load GloVe embeddings
def load_glove_model(file):
    print("Loading Glove Model")
    glove_model = {}
    with open(file, 'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model

# Replace the path with the correct path to your GloVe file
glove_path = "/home/patricia/code/ChristineSi/neuroCraft/raw_data/glove.6B.300d.txt"

# Load GloVe embeddings using your function
glove_model = load_glove_model(glove_path)


Loading Glove Model
400000 words loaded!


## Preprocessing

In [12]:
# Function for basic cleaning
def basic_cleaning(sentence):
    sentence = sentence.lower()
    sentence = ''.join(char for char in sentence if not char.isdigit())

    # Adding special quotes to the regular expression pattern
    special_quotes = '“”‘’'
    pattern = '[' + re.escape(string.punctuation + special_quotes) + ']'

    sentence = re.sub(pattern, '', sentence)
    sentence = sentence.strip()

    return sentence

# Function for tokenization
def tokenizer(sentence):
    return text_to_word_sequence(sentence)

# Function for lemmatization
def lemmatizer(sentence):
    wnl = WordNetLemmatizer()
    verbs_lemmatized = [wnl.lemmatize(word, pos='v') for word in sentence]
    nouns_lemmatized = [wnl.lemmatize(word, pos='n') for word in verbs_lemmatized]
    adverbs_lemmatized = [wnl.lemmatize(word, pos='r') for word in nouns_lemmatized]
    adj_lemmatized = [wnl.lemmatize(word, pos='a') for word in adverbs_lemmatized]
    sat_lemmatized = [wnl.lemmatize(word, pos='s') for word in adj_lemmatized]

    return sat_lemmatized

# Function for removing stopwords
def removing_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    sentence_new = [w for w in sentence if not w in stop_words]
    return sentence_new

# Apply all the steps to the training and testing text
X_train_text = [basic_cleaning(sentence) for sentence in X_train_text]
X_test_text = [basic_cleaning(sentence) for sentence in X_test_text]

X_train_text = [tokenizer(sentence) for sentence in X_train_text]
X_test_text = [tokenizer(sentence) for sentence in X_test_text]

X_train_text = [lemmatizer(sentence) for sentence in X_train_text]
X_test_text = [lemmatizer(sentence) for sentence in X_test_text]

X_train_text = [removing_stopwords(sentence) for sentence in X_train_text]
X_test_text = [removing_stopwords(sentence) for sentence in X_test_text]

# Word embedding using GloVe
def embed_sentence_glove(sentence):
    embedded_sentence = []
    for word in sentence:
        if word in glove_model:
            embedded_sentence.append(glove_model[word])

    return np.array(embedded_sentence)

def embedding_glove(sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_glove(sentence)
        embed.append(embedded_sentence)

    return embed

X_train_text_glove = embedding_glove(X_train_text)
X_test_text_glove = embedding_glove(X_test_text)

# Padding
maxlen = 105
X_train_pad_glove = pad_sequences(X_train_text_glove, dtype=float, padding='post', maxlen=maxlen)
X_test_pad_glove = pad_sequences(X_test_text_glove, dtype=float, padding='post', maxlen=maxlen)

# Function to build the NLP model with GloVe
def build_model_nlp_glove():
    model = Sequential([
        LSTM(105, input_shape=X_train_pad_glove.shape[1:]),
        Dense(64, activation='relu'),
        Dropout(0.15),
        Dense(6, activation='softmax'),
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])
    return model

# Build the NLP model with GloVe
model_nlp_glove = build_model_nlp_glove()

# Early stopping callback
es = EarlyStopping(patience=10)

# Train the model with GloVe
model_nlp_glove.fit(X_train_pad_glove, y_train,
                    validation_split=0.3,
                    epochs=50,
                    batch_size=32,
                    callbacks=[es])

# Evaluate the model with GloVe
loss_glove, accuracy_glove = model_nlp_glove.evaluate(X_test_pad_glove, y_test)
print(f'Test loss with GloVe: {loss_glove:.4f}')
print(f'Test accuracy with GloVe: {accuracy_glove:.4f}')


Epoch 1/50


2023-12-07 14:01:45.845787: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 291564000 exceeds 10% of free system memory.


73/73 [==============================] - ETA: 0s - loss: 1.7481 - accuracy: 0.2096

2023-12-07 14:02:00.709442: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 124992000 exceeds 10% of free system memory.


73/73 [==============================] - 17s 182ms/step - loss: 1.7481 - accuracy: 0.2096 - val_loss: 1.6560 - val_accuracy: 0.2681
Epoch 2/50
73/73 [==============================] - 9s 120ms/step - loss: 1.6464 - accuracy: 0.2727 - val_loss: 1.6309 - val_accuracy: 0.2742
Epoch 3/50
73/73 [==============================] - 9s 121ms/step - loss: 1.6111 - accuracy: 0.2908 - val_loss: 1.6837 - val_accuracy: 0.2540
Epoch 4/50
73/73 [==============================] - 10s 130ms/step - loss: 1.5978 - accuracy: 0.3038 - val_loss: 1.6478 - val_accuracy: 0.2601
Epoch 5/50
73/73 [==============================] - 9s 122ms/step - loss: 1.5425 - accuracy: 0.3168 - val_loss: 1.5614 - val_accuracy: 0.3115
Epoch 6/50
73/73 [==============================] - 8s 115ms/step - loss: 1.4906 - accuracy: 0.3583 - val_loss: 1.5984 - val_accuracy: 0.3196
Epoch 7/50
73/73 [==============================] - 9s 122ms/step - loss: 1.4483 - accuracy: 0.3704 - val_loss: 1.5278 - val_accuracy: 0.3367
Epoch 8/50
73/7

2023-12-07 14:04:25.183369: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 178668000 exceeds 10% of free system memory.


45/45 [==============================] - 2s 38ms/step - loss: 2.1171 - accuracy: 0.3371
Test loss with GloVe: 2.1171
Test accuracy with GloVe: 0.3371


The model with GloVe embedding perfom similar to the model with Word2Vec embedding.

## FastText embeddings

In [8]:
import fasttext


In [9]:
# Load FastText embeddings
def load_fasttext_model(file):
    print("Loading FastText Model")
    fasttext_model = fasttext.load_model(file)
    print("FastText Model Loaded!")
    return fasttext_model

# Replace the path with the correct path to your FastText model file
fasttext_model_path = "../raw_data/cc.en.300.bin"

# Load FastText embeddings using your function
fasttext_model = load_fasttext_model(fasttext_model_path)


Loading FastText Model
FastText Model Loaded!


In [11]:
# Function to embed a sentence using FastText
def embed_sentence_fasttext(model, sentence):
    embedded_sentence = []
    for word in sentence:
        # Check if the word is in the FastText model's vocabulary
        if word in model.words:
            embedded_sentence.append(model[word])

    return np.array(embedded_sentence)

# Function to create embeddings for a set of sentences
def embedding_fasttext(model, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_fasttext(model, sentence)
        embed.append(embedded_sentence)

    return embed

# Use FastText embeddings for training and testing data
X_train_text_fasttext = embedding_fasttext(fasttext_model, X_train_text)
X_test_text_fasttext = embedding_fasttext(fasttext_model, X_test_text)


KeyboardInterrupt: 

: 

In [10]:
# Padding
maxlen = 105
X_train_pad_fasttext = pad_sequences(X_train_text_fasttext, dtype=float, padding='post', maxlen=maxlen)
X_test_pad_fasttext = pad_sequences(X_test_text_fasttext, dtype=float, padding='post', maxlen=maxlen)

# Build and train the NLP model with FastText embeddings
def build_model_nlp_fasttext():
    model = Sequential([
        layers.LSTM(105, input_shape=X_train_pad_fasttext.shape[1:]),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.15),  # to prevent overfitting
        layers.Dense(6, activation='softmax'),  # for class probabilities
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])
    return model

# Build the NLP model with FastText embeddings
model_nlp_fasttext = build_model_nlp_fasttext()

# Early stopping callback
es = EarlyStopping(patience=10)

# Train the model
model_nlp_fasttext.fit(X_train_pad_fasttext, y_train,
                       validation_split=0.3,
                       epochs=50,
                       batch_size=32,
                       callbacks=[es])

# Evaluate the model
loss_fasttext, accuracy_fasttext = model_nlp_fasttext.evaluate(X_test_pad_fasttext, y_test)
print(f'Test loss with FastText: {loss_fasttext:.4f}')
print(f'Test accuracy with FastText: {accuracy_fasttext:.4f}')


KeyboardInterrupt: 